In [1]:
import Pkg; Pkg.activate(".")

include("../../TensorBenchmarks.jl")

using CairoMakie
using CliqueTrees
using DataFrames
using OMEinsumContractionOrders
using PythonCall

using OMEinsumContractionOrders: EinCode

import CSV
import KaHyPar
import Metis

using .TensorBenchmarks

const CTG = pyimport("cotengra")

const LABELS = (
    "CoTenGra",
    "OMEinsum (KaHyPar)",
    "OMEinsum (METIS)",
)

const PAIRS = (
    (Tuple{Py, Py, Py},    () -> CTG.HyperOptimizer(; reconf_opts=pydict(), max_time="rate:1e9")),
    (Tuple{EinCode, Dict}, () -> HyperND(; dis = KaHyParND(), width = 50, imbalances = 100:10:800)),
    (Tuple{EinCode, Dict}, () -> HyperND(; dis = METISND(),   width = 50, imbalances = 100:10:800)),
);

  Activating project at `~/Source/git/CliqueTreesBenchmarks.jl/tensors/omeinsum/global`
    CondaPkg Found dependencies: /Users/richardsamuelson/Source/git/CliqueTreesBenchmarks.jl/tensors/omeinsum/global/CondaPkg.toml
    CondaPkg Found dependencies: /Users/richardsamuelson/.julia/packages/CondaPkg/0UqYV/CondaPkg.toml
    CondaPkg Found dependencies: /Users/richardsamuelson/.julia/packages/PythonCall/mkWc2/CondaPkg.toml
    CondaPkg Found dependencies: /Users/richardsamuelson/.julia/environments/v1.11/CondaPkg.toml
    CondaPkg Found dependencies: /Users/richardsamuelson/.julia/packages/IJulia/Tb4Q4/CondaPkg.toml
    CondaPkg Initialising pixi
             │ /Users/richardsamuelson/.julia/artifacts/d2fecc2a9fa3eac2108d3e4d9d155e6ff5dfd0b2/bin/pixi
             │ init
             │ --format pixi
             └ /Users/richardsamuelson/Source/git/CliqueTreesBenchmarks.jl/tensors/omeinsum/global/.CondaPkg
✔ Created /Users/richardsamuelson/Source/git/CliqueTreesBenchmarks.jl/tensors/omein

In [3]:
T = Tuple{EinCode, Dict}
optimizer = HyperND(; dis = METISND(),   width = 50, imbalances = 130:1:130)
dir = joinpath("..", "..", "instances")

file = "mc_2020_017.json"
path = joinpath(@__DIR__, dir, file)
query, matrix, weights = TensorBenchmarks.read(path)
network = make(T, query, matrix, weights)

@time result = solve(network, optimizer)
tc = timecomplexity(result)
sc = spacecomplexity(result)
tc, sc
#  0.142749 seconds (1.67 M allocations: 384.175 MiB, 17.68% gc time)
# (47.0, 39.0)

  0.705054 seconds (8.34 M allocations: 6.470 GiB, 50.03% gc time)


(28.0, 23.0)